In [32]:
import pandas as pd
import re, os 
import datetime
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import warnings
warnings.filterwarnings("ignore")
os.listdir('./data_down')


"""
download data without empty ou, when select
['bca_cc_cust_rel.csv',
 'bca_without_mail.csv',
 'dwd_dsc_d365_contract_df.csv',
 'dwd_dsc_d365_opportunity_df.csv',
 'dwd_dsc_finance_estimate_summary_di.csv',
 'dwd_dsc_hr_cost_dtl.csv',
 'dwd_fact_warehouse_billing_detail_dtl.csv']

#  this doc would be on monthly & customer level
#  however the opportunity might not applicable for this.
"""

"\ndownload data without empty ou, when select\n['bca_cc_cust_rel.csv',\n 'bca_without_mail.csv',\n 'dwd_dsc_d365_contract_df.csv',\n 'dwd_dsc_d365_opportunity_df.csv',\n 'dwd_dsc_finance_estimate_summary_di.csv',\n 'dwd_dsc_hr_cost_dtl.csv',\n 'dwd_fact_warehouse_billing_detail_dtl.csv']\n\n#  this doc would be on monthly & customer level\n#  however the opportunity might not applicable for this.\n"

In [3]:
# read data
df_oppa = pd.read_csv('./data_down/' + 'dwd_dsc_d365_opportunity_df.csv', sep = '\001') 
df_esti = pd.read_csv('./data_down/' + 'dwd_dsc_finance_estimate_summary_di.csv', sep = '\001') 
df_cost = pd.read_csv('./data_down/' + 'dwd_dsc_hr_cost_dtl.csv', sep = '\001') 
df_bill = pd.read_csv('./data_down/' + 'dwd_fact_warehouse_billing_detail_dtl.csv', sep = '\001') 
df_bcaw = pd.read_csv('./data_down/' + 'bca_without_mail.csv', sep = '\001') 
rel = pd.read_csv('./data_down/' + 'bca_cc_cust_rel.csv', sep = '\001') 
df_bill_type = pd.read_csv('./data_down/' + 'dim_dsc_billing_type_info.csv', sep = '\001') 
"""
pic part data
"""
sap_fina = pd.read_csv('./data_down/' + 'smart_hr_pl_0l.csv', sep = '\001') 
levin_table = pd.read_excel('./data_down/' + 'OU_Segementation2.xlsx', sheet_name = 2) 
# for i in os.listdir('./data_down'): 
#     print(re.findall( '\w+', i)[0])
#     df = pd.read_csv('./data_down/' + i, sep = '\001') 
def colname_modi(df):
    df.columns = [ re.sub('^\w+.{1}', '', i) for i in list(df.columns)]
for i in [df_oppa, df_bill,df_cost,df_esti,rel, df_bcaw, sap_fina]:
    colname_modi(i)
del i
rel = rel.drop(['cc_in_bca', 'start_date'], axis=1)

In [2]:
# pd.set_option("display.max_rows", 15, "display.max_columns", 15)
# df_oppa[df_oppa['milestone'] == 'Closed-Lost']

# # loss_reason situation_update_next_steps description territory contract_value situation_update_next_steps sector product1
# # opportunity_type expected_golive_date	 contract_term contract_end_date

In [4]:
def data_prepare():
    """
    data prepare opportunity .
    """
    # df = pd.read_csv('./data_down/dwd_dsc_d365_opportunity_df.csv', sep = '\001')

    # df.columns = [re.sub('^\w+.{1}', '', i) for i in list(df.columns)]
    df = df_oppa.copy()
    df['contract_end_date'] = df['contract_end_date'].str.slice(0,10).fillna(pd.NaT)
    colname = ['annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue']
    for i in colname:
        df = df[~df.index.isin(df[i].str.extract('([a-zA-Z]+)').dropna().index)]

    df[['annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue']] = df[['annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue']].astype(float)
    df['contract_end_date'] = pd.to_datetime(df['contract_end_date'])

    contract_summary = df.groupby([
        'account_name_en', 'contract_end_date', 'account_id','bg', 'contract_term', 'milestone','opportunity_record'\
        ,'product1', 'product2', 'sector'
        ])[[
        'annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue', 'contract_value']].sum()
    contract_summary = contract_summary.reset_index()
    contract_summary['contract_end_yr'] = contract_summary['contract_end_date'].astype(str).str.slice(0,4)
    contract_summary['win_or_loss'] = 0
    contract_summary['win_or_loss'] = contract_summary['win_or_loss']\
        .where(contract_summary['milestone']  != 'Contract Signed', 1)
    return contract_summary

opportunity = data_prepare();opportunity.head(3)
# del contract_summary

,account_name_en,contract_end_date,account_id,bg,contract_term,milestone,opportunity_record,product1,product2,sector,annual_average_gross_profit_base,annual_average_gross_profit,annual_average_revenue_base,annual_average_revenue,contract_value,contract_end_yr
0,ABC,2021-06-28,e0209d33-e3aa-ea11-a812-000d3a851174,TSC,12.0,Closed-Canceled,New Business,Warehousing,Transportation,Technology,5000.0,5000.0,50000.0,50000.0,50000.0,2021
1,ADIDAS,2018-09-29,f64a8fa5-a304-4c85-9641-7205e77e70d8,HK,36.0,Contract Signed,New Business,Warehousing,Warehousing,Retail,950.4,1188.0,10777.6,13472.0,40416.0,2018
2,ADIDAS,2019-10-30,f64a8fa5-a304-4c85-9641-7205e77e70d8,HK,12.0,Contract Signed,Straightforward Renewals,Warehousing,Transportation,Retail,1087.2,1359.0,13123.2,16404.0,0.0,2019


In [82]:
df = df_oppa[['annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue',
        'bg', 'contract_term', 'milestone', 'opportunity_record', 'product1', 'sector', 'contract_value',
        'commission', 'fivem_customer']]  

In [83]:
# df = df[(df['milestone'] != 'Early Lead') & (df['milestone'] != 'Potential Opportunity')]
df['win_or_loss'] = 0
df['win_or_loss'] = df['win_or_loss'].where(df['milestone']  != 'Contract Signed', 1)
df = df.fillna(0)
# 删除有汉字的。
df = df[~df['annual_average_gross_profit'].str.contains('[az]').fillna(False)]

In [84]:
# df_list = [opportunity,df_bill,df_cost,df_esti]
# def check_ouname(df):
#     print(pd.Series(df.columns).str.extract('(ou_code|cost_center)').dropna(),'\t')

# for i in df_list: 
#     check_ouname(i)
# del df_list,i
import statsmodels.formula.api as smf
smf.ols('contract_value ~ contract_term', data = df).fit().summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         contract_value   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     4.123
Date:                Tue, 07 Dec 2021   Prob (F-statistic):             0.0423
Time:                        18:41:32   Log-Likelihood:                -80280.
No. Observations:                6614   AIC:                         1.606e+05
Df Residuals:                    6612   BIC:                         1.606e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept      5848.1030    582.276     10.044      0.000    4706.654    6989.551
contract_term    16.4318      8.092      2.031      0.042       0.568      32.295
==============================================================================
Omnibus:                    16857.295   Durbin-Watson:                   1.966
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        373853858.930
Skew:                          28.348   Prob(JB):                         0.00
Kurtosis:                    1166.347   Cond. No.                         75.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [85]:
# 商机金额大小与转化率的关系。 c
smf.logit(formula = "win_or_loss ~ contract_value -1 ", data = df).fit().summary()

Optimization terminated successfully.
         Current function value: 0.692395
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            win_or_loss   No. Observations:                 6614
Model:                          Logit   Df Residuals:                     6613
Method:                           MLE   Df Model:                            0
Date:                Tue, 07 Dec 2021   Pseudo R-squ.:                -0.08472
Time:                        18:41:33   Log-Likelihood:                -4579.5
converged:                       True   LL-Null:                       -4221.8
Covariance Type:            nonrobust   LLR p-value:                       nan
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
contract_value  -2.31e-06   8.67e-07     -2.665      0.008   -4.01e-06   -6.11e-07
==================================================================================
"""

In [86]:
px.violin(x = df['win_or_loss'], y =  np.log(df['contract_value']), \
    points="all", color = df['win_or_loss'], box = True )

In [87]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd

app = dash.Dash()
fig = px.violin(x = df['win_or_loss'], y =  np.log(df['contract_value']), \
    points="all", color = df['win_or_loss'], box = True )
    
app.layout = html.Div([dcc.Graph(id="life-exp-vs-gdp", figure=fig)])


if __name__ == "__main__":
    app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


SystemExit: 1

In [47]:
"""
my functions.
to customer level. functions set up.
"""
def data_clean(df, substr_year, **kwargs):
    """
    where u can set for month/year level.
    """
    df = df[~df['account_name_en'].isna()]
    df[substr_year] = df[substr_year].astype(str).str.slice(0,4) 
    df = df.groupby(['account_name_en','account_name_cn', substr_year]).agg({
        **kwargs
    })
    df = df.rename({substr_year: 'month'}, axis = 1)
    return df 

def to_cust_level(df,  substr_year, **kwargs):
    """
    (df,  substr_year, **kwargs) 必须要有ou_code or cost_center!! 
    merge all to cust level \
        to_cust_level(df,  substr_year, **kwargs):
    """
    try:
        df1 = df.merge(rel, left_on = 'ou_code', right_on = 'cc_in_ou', how  ='left')
    except:
        df1 = df.merge(rel, left_on = 'cost_center', right_on = 'cc_in_ou', how  ='left')
    return data_clean(df1, substr_year,  **kwargs)

---
# billing

In [7]:
"""
~billing~
bill detail table now having 50+ bms_types. which is unacceptalble/
"""
def data_load_bms():
    df = df_bill.copy()
    df['month'] = df['bms_bill_end_date'].astype(str).str.slice(0,6) 
    df = df.merge(df_bill_type, left_on = 'bms_fee_type', right_on = 'billing_fee_type_code', how = 'left')
    bms = df.pivot_table(index= ['cost_center', 'month'],\
        columns= 'category',
        values='bms_amount',).reset_index().fillna(0)
    return bms
bms = data_load_bms()
bms['yr'] = bms['month'].str.slice(0,4)
bms.head(2)

category,cost_center,month,人力,场地,增值,报关代理,操作,短驳运输,索赔及返利,设备使用
0,ABBOTBJXWS,202003,8153.490566,94013.547170,1368.516509,0.0,18795.386792,0.0,0.0,16845.283019
1,ABBOTBJXWS,202005,16216.742138,182958.566038,1066.343553,0.0,55997.735849,0.0,0.0,17986.415094


In [8]:
bms_cus_yr = to_cust_level(df = bms, substr_year = 'month' ,\
    cost_center = set, 
    人力 = 'sum',
    场地 = 'sum',
    增值 = 'sum',
    报关代理 = 'sum',
    操作 = 'sum',
    短驳运输 = 'sum',
    索赔及返利 = 'sum',
    设备使用 = 'sum' ).reset_index()
bms_cus_yr.head(2)

C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_21156/4278657550.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[substr_year] = df[substr_year].astype(str).str.slice(0,4)


,account_name_en,account_name_cn,month,cost_center,人力,场地,增值,报关代理,操作,短驳运输,索赔及返利,设备使用
0,ASML,阿斯麦,2020,"{ASMLXXMXAS, ASMLXSHZCS, ASMLXGZGQS, ASMLXNJLH...",1.327222e+06,2.650783e+06,277290.273019,8.301644e+05,351038.906930,3.930000e+04,0.0,218274.787736
1,ASML,阿斯麦,2021,"{ASMLXXAEPS, ASMLXXMXAS, ASMLXSHZCS, ASMLXTJXQ...",2.027038e+06,3.116845e+06,614587.514906,2.113024e+06,180360.231132,1.676826e+06,0.0,274845.181649


---
# SAP sys finance

In [48]:
def load_data_sap():
    df = sap_fina.copy()
    df = df.rename({'prctr': 'cost_center'}, axis =1 )
    return df

sap_fina = load_data_sap()

sap_cust_yr = to_cust_level(sap_fina, 'yy', \
    cost_center = set, 
    income_amt = sum,
    gross_profit_amt = sum,
    human_cost_amt = sum, 
    service_outsourcing_amt = sum,
    labor_service_outsourcing_amt = sum).reset_index()
    
sap_cust_yr = sap_cust_yr.rename({'yy':'month'}, axis =1 )

C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_21156/2950030418.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[substr_year] = df[substr_year].astype(str).str.slice(0,4)


In [425]:
def data_man_fca():
    finance_cust_ann = sap_cust_yr.merge(bms_cus_yr, on = ['account_name_en', 'month'], how = 'inner') 
    finance_cust_ann['operation_fee'] = finance_cust_ann['人力'] +  finance_cust_ann['操作']
    finance_cust_ann['bms_total'] = finance_cust_ann[['人力', '场地', '增值', '报关代理', '操作',
        '短驳运输', '索赔及返利', '设备使用']].sum(axis = 1)
    finance_cust_ann['op_pct'] = finance_cust_ann['operation_fee'] / finance_cust_ann['bms_total']
    finance_cust_ann['wh_pct'] = finance_cust_ann['场地'] / finance_cust_ann['bms_total']
    finance_cust_ann['va_pct'] = finance_cust_ann['增值'] / finance_cust_ann['bms_total']
    finance_cust_ann['type']   = finance_cust_ann[['op_pct','wh_pct','va_pct']].idxmax(axis = 1)
    return finance_cust_ann


In [302]:
def mani(type):
    df = finance_cust_ann[finance_cust_ann['type'] == type] 
    df['flag'] = df[i] < .5
    return df
finance_cust_ann2 = pd.DataFrame()
for i in finance_cust_ann['type'].unique():
    finance_cust_ann2 = pd.concat([finance_cust_ann2, mani(i)], axis = 0)
finance_cust_ann2.loc[finance_cust_ann2['flag'] == True,  'type'] = 'balance'

finance_cust_ann2.head(1)

C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_21156/2853613689.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,account_name_en,account_name_cn_x,month,cost_center_x,income_amt,gross_profit_amt,human_cost_amt,service_outsourcing_amt,labor_service_outsourcing_amt,account_name_cn_y,...,短驳运输,索赔及返利,设备使用,bms_total,operation_fee,op_pct,wh_pct,va_pct,type,flag
0,ASML,阿斯麦,2020,"{ASMLXXMXAS, ASMLXSHZCS, ASMLXGZGQS, ASMLXTJXQ...",44186486.9,7216033.92,8542221.21,136642.98,249772.05,阿斯麦,...,39300.0,0.0,218274.787736,5.694073e+06,1.678261e+06,0.294738,0.465534,0.048698,balance,True


In [303]:
finance_cust_ann2 = finance_cust_ann2.drop('flag', axis = 1)
finance_cust_ann2['type'] = finance_cust_ann2['type'].replace(
    {'wh_pct': '仓储型', 'balance': '平衡型', 'va_pct': '增值型', 'op_pct': '操作型'})

In [304]:
finance_cust_ann2['account_name_en'].nunique()

# my mapping 116 customers.

116

In [305]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [306]:
# finance_cust_ann2.to_csv('./plot_data.csv', index = None, encoding = 'utf_8_sig', sep = '\001')
plot_data = finance_cust_ann2[finance_cust_ann2['month'] > "2019"]

In [307]:
plot_data = plot_data.rename({'month' : 'year'}, axis = 1)

In [334]:
plot_data['profit_rate'] = plot_data['gross_profit_amt']/ (plot_data['income_amt'] - plot_data['gross_profit_amt'])
plot_data = plot_data[plot_data['profit_rate'] < 3]

In [381]:
cust_weight_avg  = plot_data.groupby('year').apply(
    lambda x: np.average(x['profit_rate'], weights=x['income_amt']))
line_20 = cust_weight_avg[0].round(4)
line_21 = cust_weight_avg[1].round(4)


In [408]:
 
fig = px.scatter(
    data_frame= plot_data, x = 'income_amt', y= 'profit_rate', hover_data=['account_name_cn_x'],
    size = 'income_amt', color = 'type', facet_col = 'year', title='profit_rate against income, customer level')  
# hover_data=  ['account_name_en'] , 
fig.add_hline(y = line_20, line_width=3, line_dash="dash", line_color="black", opacity = .4,\
    annotation_text="2020 <br> weighted mean profit_rate <br>, %s" %line_20 , 
    annotation_position="top right", annotation_opacity = .4, col = 1)
fig.add_hline(y = line_21, line_width=3, line_dash="dash", line_color="black", opacity = .4,\
    annotation_text="2021 <br> weighted mean profit_rate <br>, %s" %line_21 , 
    annotation_position="top right", annotation_opacity = .4, col = 2)

In [409]:
fig.write_html('./fig_cust_profit_rate.html')
# plot_data.to_csv('./data_out/income_customer_cats.csv', encoding = 'utf_8_sig', index = None)

In [285]:
bms_cc_ou = bms.groupby(['cost_center','yr']).agg({
    '人力': sum, 
    '场地': sum, 
    '增值': sum, 
    '报关代理': sum, 
    '操作': sum, 
    '短驳运输': sum, 
    '索赔及返利': sum,
    '设备使用': sum,
}).reset_index()

sap_cc_ou = sap_fina.groupby(['yy', 'cost_center']).agg({
    'income_amt': sum, 
    'gross_profit_amt': sum,
    'human_cost_amt': sum,
    'service_outsourcing_amt': sum,
    'labor_service_outsourcing_amt': sum,
}).reset_index()

sap_cc_ou = sap_cc_ou.rename({'yy': 'yr'}, axis = 1)

bms_cc_ou['yr'] = bms_cc_ou['yr'].astype(int)

In [293]:
plt2 = sap_cc_ou.merge(bms_cc_ou, on = ['cost_center', 'yr'], how = 'inner')
plt2 = plt2[plt2['yr'] > 2019]

In [295]:
plt2.shape # 560

(560, 15)

In [296]:
def data_man_cc(finance_cust_ann): 
    """
    客户的收益类型.计算
    """
    finance_cust_ann['operation_fee'] = finance_cust_ann['人力'] +  finance_cust_ann['操作']
    finance_cust_ann['bms_total'] = finance_cust_ann[['人力', '场地', '增值', '报关代理', '操作',
        '短驳运输', '索赔及返利', '设备使用']].sum(axis = 1)
    finance_cust_ann['op_pct'] = finance_cust_ann['operation_fee'] / finance_cust_ann['bms_total']
    finance_cust_ann['wh_pct'] = finance_cust_ann['场地'] / finance_cust_ann['bms_total']
    finance_cust_ann['va_pct'] = finance_cust_ann['增值'] / finance_cust_ann['bms_total']
    finance_cust_ann['type']   = finance_cust_ann[['op_pct','wh_pct','va_pct']].idxmax(axis = 1)
    return finance_cust_ann
plt2 = data_man_cc(plt2)
plt2 = plt2[~plt2['type'].isna()]
# 560 

In [329]:
def mani(type):
    """
    add balance type to the table.
    """
    df = plt2[plt2['type'] == type] 
    df['flag'] = df[type] < .5
    return df
plt21 = pd.DataFrame()
for i in plt2['type'].unique():
    plt21 = pd.concat([plt21, mani(i)], axis = 0)
plt21.loc[plt21['flag'] == True,  'type'] = 'balance'

plt21 = plt21.drop('flag', axis = 1)
plt21['type'] = plt21['type'].replace(
    {'wh_pct': '仓储型', 'balance': '平衡型', 'va_pct': '增值型', 'op_pct': '操作型'})
 

C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_21156/2659271940.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



343

In [345]:
plt21['profit_rate'] = plt21['gross_profit_amt']/ (plt21['income_amt'] - plt21['gross_profit_amt'])
plt21 = plt21[(plt21['profit_rate'] < 2) & (plt21['profit_rate'] > -2)]

C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_21156/2140466091.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [421]:
cc_weight_avg[2021]

0.14562988090173806

In [422]:
cc_weight_avg  = plt21.groupby('yr').apply(
    lambda x: np.average(x['profit_rate'], weights=x['income_amt']))
line_20 = cc_weight_avg[2020].round(4)
line_21 = cc_weight_avg[2021].round(4)


In [423]:
 
fig2 = px.scatter(
    data_frame= plt21, x = 'income_amt', y= 'profit_rate', hover_data=['cost_center'],
    size = 'income_amt', color = 'type', facet_col = 'yr', title='profit margin against income, cost_center level')  
# hover_data=  ['account_name_en'] , 


fig2.add_hline(y = line_20, line_width=3, line_dash="dash", line_color="black", opacity = .4,\
    annotation_text="2020 <br> weighted mean profit_rate <br>, %s" %line_20 , 
    annotation_position="top right", annotation_opacity = .4, col = 1)
fig2.add_hline(y = line_21, line_width=3, line_dash="dash", line_color="black", opacity = .4,\
    annotation_text="2021 <br> weighted mean profit_rate <br>, %s" %line_21 , 
    annotation_position="top right", annotation_opacity = .4, col = 2)

In [424]:
fig2.write_html('./fig_cc_profit_rate.html')

---
# cost

In [ ]:
# def tst(substr_year,**kwargs):
#     df_bill[substr_year] = df_bill[substr_year].astype(str).str.slice(0,4)
#     return  df_bill.groupby([substr_year, 'cost_center']).agg({
#         **kwargs
#     }).reset_index()

# tst(substr_year = 'bms_bill_end_date' ,\
#     bms_amount = 'sum', bms_fee_type = [set,'nunique'], bms_fee_name = set, bms_bill_id = 'nunique')

# test = df_bill.head(39)
# test.pivot_table(index= ['bms_bill_id', 'cost_center'],\
#     columns= ['bms_fee_type'],
#     values='bms_amount',)

In [ ]:
"""
# cost
"""
def data_load_cost():
    df = df_cost
    df = df.dropna(axis=1, how = 'all').fillna(0)
    frame = pd.DataFrame(df.sum(axis = 0) == 0)
    df = df.loc[:, frame[frame[0] == False].index.tolist()]
    df['year'] = df['month'].astype(str).str.slice(0,4)
    return df


df_cost = data_load_cost()
df_cost.head(2)

In [ ]:
cost_cus_yr = to_cust_level(df_cost, 'month',\
     rev = 'sum',
     gp = 'sum',
     hr_cost  = 'sum', 
     hc  = 'sum',
     hc_os  = 'sum',
     total_working_hrs  = 'sum',
     staff_cost_fixed  = 'sum',
     staff_cost_vari  = 'sum',
     staff_cost_os_fixed  = 'sum',
     staff_cost_os_vari  = 'sum',
     staff_cost_bizos_actual  = 'sum'
     ).reset_index()
cost_cus_yr.head()

---

---
# merge

In [ ]:
bms_cus_yr['account_name_en'].nunique()

In [ ]:
test = bms_cus_yr.merge(cost_cus_yr, on = ['account_name_en', 'month'], how = 'inner')

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
test.account_name_en.nunique()

In [ ]:
test['account_name_en'].nunique()

In [ ]:
# test[test['account_name_en'].str.lower().str.match('bose')]

In [ ]:
# df_oppa[df_oppa['account_name_en'].str.lower().str.match('bmw').fillna(False)][['bg','opportunity_number', 'opportunity_name', 'account_id',
#        'account_name_en', 'account_name_cn', 'actual_close_date',
#        'annual_average_gross_profit_base', 'annual_average_gross_profit',
#        'annual_average_revenue_base', 'annual_average_revenue',
#        'bca_ref_number', 'bca_status']]

In [ ]:
# fig = go.Figure(data = [go.Scatter(x = df_sub ['contract_end_date'] ,
#                                 y = df_sub ['annual_average_revenue'],
#                                 mode='markers',
#                                 marker_size=df_sub['annual_average_revenue']/1000,
#                                 opacity=0.87,
#                                 name='account',
#                                 textposition='bottom center')],
#                                 layout=go.Layout(
#                                     xaxis = dict(range = [min, max]),
#                                     yaxis = dict(range = [0, 100000]),
#                                     legend = {'xanchor': 'left'}
#                                 )
#                                 ) 
# fig

In [ ]:
# df['contract_end_date'] = df['contract_end_date'].str.slice(0,10).fillna(pd.NaT)
# colname = ['annual_average_gross_profit_base', 'annual_average_gross_profit',
#        'annual_average_revenue_base', 'annual_average_revenue']
# for i in colname:
#     df = df[~df.index.isin(df[i].str.extract('([a-zA-Z]+)').dropna().index)]

# df[['annual_average_gross_profit_base', 'annual_average_gross_profit',
#     'annual_average_revenue_base', 'annual_average_revenue']] = df[['annual_average_gross_profit_base', 'annual_average_gross_profit',
#     'annual_average_revenue_base', 'annual_average_revenue']].astype(float)
# df['contract_end_date'] = pd.to_datetime(df['contract_end_date'])
# contract_summary = df.groupby(['account_name_en', 'contract_end_date'])[[
#        'annual_average_gross_profit_base', 'annual_average_gross_profit',
#        'annual_average_revenue_base', 'annual_average_revenue']].sum()
# contract_summary = contract_summary.reset_index()
# contract_summary['contract_end_yr'] = contract_summary['contract_end_date'].astype(str).str.slice(0,4)

In [ ]:
contract_summary['contract_end_date']

In [ ]:
# fig = px.scatter(x = "contract_end_date", y= "annual_average_revenue", \
#     color = 'account_name_en',  size = 'annual_average_revenue',
#     data_frame  = contract_summary.head(65))

# min = pd.to_datetime('2018-01-01', format='%Y-%m-%d')
# max = pd.to_datetime('2024-01-01', format='%Y-%m-%d')
# fig.update_xaxes(range = list([min, max]))

# fig.update_layout(
#     title_text="Title",
#     showlegend = True
# )

# # Add range slider
# fig.update_layout(

# )

